## Rekognition Custom Label Introduction
With Amazon Rekognition Custom Labels, you can identify the objects and scenes in images that are specific to your business needs. For example, you can find your logo in social media posts, identify your products on store shelves, classify machine parts in an assembly line, distinguish healthy and infected plants etc.

Rekognition Custom Labels builds off of Rekognition’s existing capabilities, which are already trained on tens of millions of images across many categories. So, it uses transfer learning to build the ML model. 

<br />

![]url(./1a.PNG)

## Necessary set-up

1. Once you are loggged in to the console, first step is create a S3 bucket where you will upload the images. 
2. Download the images from the google drive link https://drive.google.com/drive/folders/1Z_B1BDM9x9PdJ7A226_MWyKQ-DMj34Tf?usp=sharing and unzip it.

3. Create a S3 bucket and upload images
    - Type "s3" in the search bar and open the S3 bucket console. 
    - Create a S3 bucket with a unique name. The names have to be globally unique and all lower cap.
    - Example: '2022-nimblevision-chinnayya-waterlevel' and '2022-nimblevision-anu-waterlevel'
    - Open the S3 bucket that you created and choose "upload" > "Add Folder". select Level_50 and Level_75 folders one by one.
    - Verify that the images are successfully uploaded in corresponding folders (50 and 75)
    
4. Open the manifest file and change the s3 bucket name rto yo was sent over email to the same S3 bucket that you created
    - Import the manifest file to the same S3 bucket that you created

5. Create a SageMaker notebook instance.
    - open SageMaker console in another tab and under images, click on "Notebook" > "Notebook Instances"
    - Click on "create notebook instance"
    - Give a notebook instance name. Don't use any symbol in the name.
    - Under "Permissions and encryption", create a new IAM role.
    - Then create new instance. It will take few min to create the instance.

## Rekognition Custom Label Dataset Creation

Choose custom label and click "Get started" 
   - if you are using rekognition custom label for the first time, it will prompt you to create a S3 bucket. Please create it. The bucket will be automatically created by custom label and all the labelling and modelling artifacts will be stored here.
    
   - Click "Projects" > "Create project". Give project name and create project. Once the project is created you should see the below screen
   
   - Click on "Create dataset" and select "Start with a single dataset" 
   
   - Select "Import images labeled by SageMaker Ground Truth" 
   

<br />

![](./2a.png)
   
   
   
   - Go to your S3 bucket. Select (tick) the manifest file, copy the S3 URI and paste it in the S3 URI location in console.
   
   - Create the dataset
   
   - Now you should see the images with boxes drawn around the float switch

## Rekognition Custom Label Model Training

1. Now click on "Train model". Model training will take ~1 hour. In the console, if you select your project, you should see a status like TRAINING_IN_PROGRESS. 

## Rekognition Custom Label Model Metric Verification

Once the model is traine, you should be able to see the results


<br />

![Flow](./3a.png)



## Testing the model for inference

Now you can open JupyterLab and import the notebook.

In [ ]:
# Import libraries and necessary set-up

import boto3
import io
from IPython.display import HTML, display, Image as IImage
from PIL import Image, ImageDraw, ImageFont
import json
import math

client=boto3.client('rekognition')
s3 = boto3.client('s3')
s3_connection = boto3.resource('s3')

bucket='rekognition-projects-2022'

## Start The Model

You can start the model from the console. You can select "use model" tab and click on "Start model"

In [ ]:
# Model can also be started by using the below code, but we will ignore it for now.

import boto3

def start_model(project_arn, model_arn, version_name, min_inference_units):

    client=boto3.client('rekognition')

    try:
        # Start the model
        print('Starting model: ' + model_arn)
        response=client.start_project_version(ProjectVersionArn=model_arn, MinInferenceUnits=min_inference_units)
        # Wait for the model to be in the running state
        project_version_running_waiter = client.get_waiter('project_version_running')
        project_version_running_waiter.wait(ProjectArn=project_arn, VersionNames=[version_name])

        #Get the running status
        describe_response=client.describe_project_versions(ProjectArn=project_arn,
            VersionNames=[version_name])
        for model in describe_response['ProjectVersionDescriptions']:
            print("Status: " + model['Status'])
            print("Message: " + model['StatusMessage']) 
    except Exception as e:
        print(e)
        
    print('Done...')
    
def main():
    project_arn='arn:aws:rekognition:us-east-1:462768798410:project/water-level/1643853256248'
    model_arn='arn:aws:rekognition:us-east-1:462768798410:project/water-level/version/water-level.2022-02-03T15.23.44/1643873024451'
    min_inference_units=1 
    version_name='water-level.2022-02-03T15.23.44'
    start_model(project_arn, model_arn, version_name, min_inference_units)

if __name__ == "__main__":
    main()

## Import Test Images from S3

In [ ]:
#img = "water-level-detection/Level_50/IMG_0402.JPG" #level 50
img = "water-level-detection/Level_75/IMG_0281.JPG" #level 75
#img = "water-level-detection/Level_75/IMG_0297.JPG" #level 75
#img = "water-level-detection/Level_50/IMG_0410.JPG" #level 50

ratio_tankheight_bbdiag = 0.261 # ratio of actual tank height to the bounding box diagonal

display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucket, 'Key': img}), width=800))

## Analyze Image

In [ ]:
def display_image(bucket,img,response):
    # Load image from S3 bucket

    s3_object = s3_connection.Object(bucket,img)
    s3_response = s3_object.get()

    stream = io.BytesIO(s3_response['Body'].read())
    image=Image.open(stream)

    # Ready image to draw bounding boxes on it.
    imgWidth, imgHeight = image.size
    draw = ImageDraw.Draw(image)
    
  
    # calculate and display bounding boxes for each detected custom label
    for customLabel in response['CustomLabels']:
 
        if 'Geometry' in customLabel:
            box = customLabel['Geometry']['BoundingBox']
            left = imgWidth * box['Left']
            top = imgHeight * box['Top']
            width = imgWidth * box['Width']
            height = imgHeight * box['Height']
            diag = round(math.sqrt(width**2+height**2))
            waterlevel = round(diag*ratio_tankheight_bbdiag)
            print("water level is at approx.: ", waterlevel, " cm")

            fnt = ImageFont.truetype("/usr/share/fonts/dejavu/DejaVuSans-Bold.ttf", 65)
            # use red for level 50 and blue for level 75
            if (customLabel['Name'] == "Level_50"):
                fontColor = "red"
            else:
                fontColor = "blue"
                
            draw.text((left,top-100), customLabel['Name'], fontColor, font=fnt)

            points = (
                (left,top),
                (left + width, top),
                (left + width, top + height),
                (left , top + height),
                (left, top))
            draw.line(points, fill='#00d400', width=5)

    image.show()
    

def show_custom_labels(model,bucket,img, min_confidence):

    # Call DetectCustomLabels
    response = client.detect_custom_labels(Image={'S3Object': {'Bucket': bucket, 'Name': img}},
        MinConfidence=min_confidence,
        ProjectVersionArn=model)
    
    # display image.
    display_image(bucket,img,response)

    #return the json response
    return (json.dumps(response['CustomLabels'], indent=2))

def main():

    model='arn:aws:rekognition:us-east-1:462768798410:project/water-level/version/water-level.2022-02-03T15.23.44/1643873024451'
    min_confidence=50

    label_count=show_custom_labels(model,bucket,img, min_confidence)
    print("Custom labels detected: " + str(label_count))


if __name__ == "__main__":
    main()

In [ ]:
# Function to estimate water level to bounding box diag ratio based on the images from a S3 folder containing images

def ratio_tankheight_bbdiag(bucket,key, level):
    paginator = s3.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=BUCKET, Prefix=KEY)
    sum=0
    count=0

    for page in pages:
        for obj in page['Contents']:
            img=obj['Key']
            s3_object = s3_connection.Object(BUCKET,img)
            s3_response = s3_object.get()

            stream = io.BytesIO(s3_response['Body'].read())
            image=Image.open(stream)

            imgWidth, imgHeight = image.size
            response = client.detect_custom_labels(Image={'S3Object': {'Bucket': bucket, 'Name': img}}, MinConfidence=min_confidence, ProjectVersionArn=model)
            for customLabel in response['CustomLabels']:

                if 'Geometry' in customLabel:
                    box = customLabel['Geometry']['BoundingBox']
                    left = imgWidth * box['Left']
                    top = imgHeight * box['Top']
                    width = imgWidth * box['Width']
                    height = imgHeight * box['Height']
                    diag = round(math.sqrt(width**2+height**2))
                    sum+=diag
                    count+=1
                    avgdiag=sum/count               
    ratio=(tank_height*level/(100*avgdiag))
    return(ratio)

In [ ]:
# Estimate water level to bounding box diag ratio for level 50

BUCKET = 'rekognition-projects-2022'
KEY = 'water-level-detection/Level_50/'
tank_height = 150
level=50
model='arn:aws:rekognition:us-east-1:462768798410:project/water-level/version/water-level.2022-02-03T15.23.44/1643873024451'
min_confidence=50
model_arn='arn:aws:rekognition:us-east-1:462768798410:project/water-level/version/water-level.2022-02-03T15.23.44/1643873024451'

ratio_tankheight_bbdiag(BUCKET,KEY,level)

In [ ]:
# Estimate water level to bounding box diag ratio for level 75

BUCKET = 'rekognition-projects-2022'
KEY = 'water-level-detection/Level_75/'
level=75
model='arn:aws:rekognition:us-east-1:462768798410:project/water-level/version/water-level.2022-02-03T15.23.44/1643873024451'
model_arn='arn:aws:rekognition:us-east-1:462768798410:project/water-level/version/water-level.2022-02-03T15.23.44/1643873024451'

ratio_tankheight_bbdiag(BUCKET,KEY,level)

In [ ]:
# we need to get more levels to get an accurate estimate of the ratio of water level to bounding box diag